In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./bda_eval.csv')

In [4]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
model = api.load("word2vec-google-news-300")
lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[=================================================-] 99.8% 1659.9/1662.8MB downloaded


In [5]:
df.values[1:, 3:] = df.values[0, 3:]

In [6]:
def get_scores(f, alpha = 1, beta = 1):
  ctr = 0
  for i in df.values:
    ctr+=1
    anc = i[0]
    pos = list(i[[1, 2]])
    neg = list(i[3:])
    anc = f(anc)
    pos = list(map(f, pos))
    neg = list(map(f, neg))
    anc_dot = lambda x: np.array(anc).T @ np.array(x)
    pos = list(map(anc_dot, pos))
    neg = list(map(anc_dot, neg))
    print("data", ctr, "score = ", alpha*sum(pos)/2 - beta*sum(neg)/10)

In [7]:
import string
def preprocess(x):
  x = str(x)
  x = x.lower()
  x = "".join([char for char in x if char not in string.punctuation])
  words = word_tokenize(x)
  words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  x = ' '.join(words)
  return x

In [8]:
import numpy as np
def word2vec_f(x):
  x = preprocess(x)
  x = x.split(' ')
  embs = []
  for i in x:
    try:
      embs.append(model[i])
    except:
      pass
  x = np.array(embs)
  x = np.mean(x, axis = 0)
  if(embs == []):
    x = np.zeros((300, ))
  # print(x.shape)
  return x

In [9]:
get_scores(word2vec_f, 1, 1)

data 1 score =  0.302653044462204
data 2 score =  0.19405337870120998
data 3 score =  0.2951394438743591
data 4 score =  0.20411428809165955


In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.6 MB/s eta 0:00:00


In [11]:
import numpy as np
import pandas as pd
import torch as nn
import transformers 
import json
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def bert_f(x):
  x = preprocess(x)
  inp = nn.tensor(tokenizer.encode(x))[:512]
  inp = inp.unsqueeze(0)
  outputs = model(inp)
  return np.array(outputs[0].detach()[0][0])

In [13]:
get_scores(bert_f, 1, 1)

data 1 score =  16.87805633544923
data 2 score =  4.666716003417974
data 3 score =  15.89978485107423
data 4 score =  0.8125961303711051


In [14]:
model_glove = api.load("glove-twitter-200")

def glove_f(x):
  x = preprocess(x)
  x = x.split(' ')
  embs = []
  for i in x:
    try:
      embs.append(model_glove[i])
    except:
      pass
  x = np.array(embs)
  x = np.mean(x, axis = 0)
  if(embs == []):
    x = np.zeros((300, ))
  # print(x.shape)
  return x

[=================================================-] 99.9% 757.4/758.5MB downloaded


In [15]:
get_scores(glove_f, 1, 1)

data 1 score =  2.3014987945556644
data 2 score =  2.698239326477051
data 3 score =  1.8718524932861325
data 4 score =  -0.43235392570495534
